# 畳み込みニューラルネットワーク

## 学習済みデータを使った予測

### モジュールの読み込み

In [ ]:
from __future__ import print_function
import argparse
import datetime
import json
import multiprocessing
import random
import sys
import threading
import time

import numpy as np
from PIL import Image

import six
#import six.moves.cPickle as pickle
import cPickle as pickle
from six.moves import queue

import chainer
import matplotlib.pyplot as plt
import numpy as np
import math
import chainer.functions as F
import chainer.links as L
from chainer.links import caffe
from matplotlib.ticker import *
from chainer import serializers

import os
#sys.path.append(os.getcwd() + '/ch3')
import nin
import alex

### モデルの読み込み

モデルを読み込みます

In [ ]:
model = alex.Alex()

In [ ]:
serializers.load_npz("result_alex/model_iter_15000", model)
model.to_cpu()

### 画像の読み込み

判定する画像を読み込む関数を作成します。
* 入力画像のサイズ（256×256）をモデルの入力サイズ（227×227）へ変換します。
* 画像の数値を０~２５５を０~１の範囲に変換します。

In [ ]:
model.insize

In [ ]:
cropwidth = 256 - model.insize
mean_image = np.load("mean.npy")

In [ ]:
def read_image(path, center=False, flip=False):
  image = np.asarray(Image.open(path)).transpose(2, 0, 1)
  if center:
    top = left = cropwidth / 2
  else:
    top = random.randint(0, cropwidth - 1)
    left = random.randint(0, cropwidth - 1)
  bottom = model.insize + top
  right = model.insize + left
  image = image[:, top:bottom, left:right].astype(np.float32)
  image -= mean_image[:, top:bottom, left:right]
  image /= 255
  if flip and random.randint(0, 1) == 0:
    return image[:, :, ::-1]
  else:
    return image

判定する画像を読み込み、chainer.Variableへ変換します。

In [ ]:
img = read_image("what_is_this_1.jpg",center=True)

In [ ]:
x = np.ndarray((1, 3, model.insize, model.insize), dtype=np.float32)
x[0]=img
x = chainer.Variable(np.asarray(x))

### 画像カテゴリの予測

モデルと入力画像から予測を行います。

In [ ]:
def predict_alex(net,x):
    h = F.max_pooling_2d(F.local_response_normalization(
            F.relu(net.conv1(x))), 3, stride=2)
    h = F.max_pooling_2d(F.local_response_normalization(
            F.relu(net.conv2(h))), 3, stride=2)
    h = F.relu(net.conv3(h))
    h = F.relu(net.conv4(h))
    h = F.max_pooling_2d(F.relu(net.conv5(h)), 3, stride=2)
    h = F.dropout(F.relu(net.fc6(h)))
    h = F.dropout(F.relu(net.fc7(h)))
    h = net.fc8(h)

    return F.softmax(h)

In [ ]:
with chainer.using_config('train', False):
    with chainer.no_backprop_mode():
        score = predict_alex(model,x)

In [ ]:
categories = np.loadtxt("labels.txt", str, delimiter="\t")

In [ ]:
top_k = 20
prediction = zip(score.data[0].tolist(), categories)
prediction.sort(cmp=lambda x, y: cmp(x[0], y[0]), reverse=True)
for rank, (score, name) in enumerate(prediction[:top_k], start=1):
    print('#%d | %s | %4.1f%%' % (rank, name, score * 100))

In [ ]:
categories

## numpyの基礎

ニューラルネットワークのプログラミングには、numpyによる計算が多く登場します。今回の授業に関連するものについて、簡単に補足します。

### モジュールのインポート

numpyを使うにには、最初にモジュールをインポートする必要があります。

In [ ]:
import numpy as np

### 配列の作成

配列は、ndarrayとよばれるオブジェクトで表現します。ndarrayはpythonのリストから作成します。

In [ ]:
listA = [[0,1,2],[3,4,5],[6,7,8]]
listA

リストから配列を作成

In [ ]:
arrayA = np.array(listA)
arrayA

このように書いても同じです。

In [ ]:
arrayA = np.array([[0,1,2],[3,4,5],[6,7,8]])
arrayA

配列が数列の場合(np.arange)

10から始まって、20未満の、2つとびの数列から配列を作成

In [ ]:
arrayB = np.arange(10,20,2)
arrayB

配列の要素が０の場合(np.zeros)

In [ ]:
arrayZ = np.zeros(5)
arrayZ

In [ ]:
arrayZ2 = np.zeros((2,3))
arrayZ2

### 配列の形状の変更

reshapeメソッドで配列の形状を変更できます。

In [ ]:
arrayC = np.arange(0,9)
arrayC

In [ ]:
arrayC = arrayC.reshape(3,3)
arrayC

In [ ]:
arrayD = np.array([0,1,2,3,4,5]).reshape(2,3)
arrayD

１０×１０の画像（赤、緑、青の３成分で構成されるもの）が４枚ある場合、どの次元がどの部分に該当するか注意する必要があります。処理系により異なることがあります。

In [ ]:
arrayE = np.zeros(1200).reshape(4,3,10,10)
arrayE

### 配列の形状の確認(shape)

先に作成した画像データの配列について、形状を確認してみましょう。

In [ ]:
arrayE.shape

### 配列のデータタイプ

In [ ]:
arrayA = np.array([[0,1,2],[3,4,5],[6,7,8]])
arrayA

データタイプを調べる

In [ ]:
arrayA.dtype

データタイプを指定して配列を作成

In [ ]:
arrayF = np.array([[0,1,2],[3,4,5],[6,7,8]],dtype='f')
arrayF

In [ ]:
arrayF.dtype

## Alexnetの仕組み

### alex.pyのソースコード

In [ ]:
import numpy as np

import chainer
import chainer.functions as F
from chainer import initializers
import chainer.links as L


class Alex(chainer.Chain):

    """Single-GPU AlexNet without partition toward the channel axis."""

    insize = 227

    def __init__(self):
        super(Alex, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None,  96, 11, stride=4)
            self.conv2 = L.Convolution2D(None, 256,  5, pad=2)
            self.conv3 = L.Convolution2D(None, 384,  3, pad=1)
            self.conv4 = L.Convolution2D(None, 384,  3, pad=1)
            self.conv5 = L.Convolution2D(None, 256,  3, pad=1)
            self.fc6 = L.Linear(None, 4096)
            self.fc7 = L.Linear(None, 4096)
            self.fc8 = L.Linear(None, 1000)

    def __call__(self, x, t):
        h = F.max_pooling_2d(F.local_response_normalization(
            F.relu(self.conv1(x))), 3, stride=2)
        h = F.max_pooling_2d(F.local_response_normalization(
            F.relu(self.conv2(h))), 3, stride=2)
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = F.max_pooling_2d(F.relu(self.conv5(h)), 3, stride=2)
        h = F.dropout(F.relu(self.fc6(h)))
        h = F.dropout(F.relu(self.fc7(h)))
        h = self.fc8(h)

        loss = F.softmax_cross_entropy(h, t)
        chainer.report({'loss': loss, 'accuracy': F.accuracy(h, t)}, self)
        return loss


まず、a = Alex()
として、インスタンスを生成した場合に、\_\_init\_\_メソッドが呼び出されます。そして、作成したインスタンスを、
loss = a(x,t)
として呼び出した時に、\_\_call\_\_メソッドが呼び出されます。

### Convolution2D

テスト用にRBGの三色からなる10×10の画像を１枚作成します。

In [ ]:
import numpy as np
x = np.arange(-1 * 3 * 10 * 10 / 2,1 * 3 * 10 * 10 / 2, dtype='f').reshape(1, 3, 10, 10) 

In [ ]:
x.shape

In [ ]:
x

入力チャネル数が3、出力チャネル数が7、カーネルの大きさは5×5の Convolution2Dのレイヤーを作成します。Lはchainer.linksを表します。chainer.linkは学習のためのウエイトを含みます。

In [ ]:
import chainer.links as L
l = L.Convolution2D(3, 7, 5) 

作成したレイヤーlに画像を入力して、出力データの形を確認します。

In [ ]:
y = l(x)
y.shape

画像のサイズは６×６、チャネル数は７になっています。

### Relu関数

Relu関数の形をみてみましょう。

In [ ]:
import numpy as np
x_test = np.arange(-8, 8, 1, dtype='f')

In [ ]:
x_test

In [ ]:
import chainer.functions as F
y_test = F.relu(x_test)

In [ ]:
y_test.data

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_test, y_test.data, lw=5) # プロット
plt.xlim(-8, 8)  # x軸の範囲
plt.ylim(-2, 8) # y軸の範囲
plt.grid() # グリッド描画
plt.show() # グラフを出力

In [ ]:
F.relu(x)

### シグモイド関数

シグモイド関数の形を見てみましょう。

In [ ]:
import chainer.functions as F
y_test = F.sigmoid(x_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_test, y_test.data, lw=5) # プロット
plt.xlim(-8, 8)  # x軸の範囲
plt.ylim(0, 1) # y軸の範囲
plt.grid() # グリッド描画
plt.show() # グラフを出力

シグモイド関数の特徴は、関数の範囲が０から１までということです。また、グラフをみると判りますが、入力される値が非常に大きい場合、あるいは、非常に小さい場合には、シグモイド関数の傾きは非常に小さくなります。したがって、シグモイド関数の前の層から出力される値がそのような場合には、前の層の値が変化しても、シグモイド関数から出力される値の変化は非常に小さくなります。

ReLU関数の場合には、前の層から出力される値がプラスであれば、後の層に信号の変化がそのまま伝達されるのに対して、シグモイド関数の場合には、信号の変化を有効に後の層へ伝えることができる入力信号の値の範囲がReLUよりも狭いと言えます。

また、ReLU関数の場合には、信号が伝わる場合には、その傾きが１であるため、信号の変化がそのまま後の層に伝わります。標準的なシグモイド関数の場合、その傾きの最大値は０．２５です。したがって、前の層の出力が１変化しても、後の層の信号の変化が最大でもその４分の１に弱められます。そのため、シグモイド関数をつかって複数の階層をつなげていくと、入力層に近い層のウエイトを変化させても、出力層へ出力される値の変化がほどんどなくなってしまいます。これを勾配の消失といいます。

このような問題があるため、シグモイド関数は中間層でなく、出力層の活性化関数として、最終的に出力される値を０から１の範囲に調整するために用いられます。

## max_pooling_2d

In [ ]:
x.shape

画像の大きさは１０×１０です。フィルタの大きさが３×３ ストライドが１のmax_pooling_2dをかけます。

In [ ]:
mx = F.max_pooling_2d(x,3,stride=1)

In [ ]:
mx.shape

画像の大きさは８×８となっています。

In [ ]:
x_test2 = np.array([[1,1,2,4,],[5,6,7,8],[3,2,0,1],[1,2,3,4]], dtype='f').reshape(1,1,4,4)

In [ ]:
x_test2

In [ ]:
y_test2 = F.max_pooling_2d(x_test2,2,stride=2)

In [ ]:
y_test2.data

### dropout

学習中に一定の確率ratioでランダムに入力要素を振るい落とします。そして残った要素を$\frac{1}{ratio}$倍します。学習中でない場合には、単に入力された値をそのまま返します。

ドロップアウトを行うことにより、学習時にネットワークの自由度を強制的に小さくして汎化性能を上げ、過学習を避けることができます。

In [ ]:
x

In [ ]:
F.dropout(x) #学習中の場合

In [ ]:
#学習中でない場合　入力された値をそのまま返す
import chainer
with chainer.using_config('train', False):
    y = F.dropout(x)

In [ ]:
y

### 全結合層 (fully-connected layer)

<img src="fullyconnected3.png">

入力データを作成します

In [ ]:
x_test3 = np.array([[0, 1, 2, 3, 4]], np.float32)

In [ ]:
x_test3.shape

入力のサイズが５、出力のサイズが１０の全結合層を作成します。

In [ ]:
l = L.Linear(5, 10)

In [ ]:
y_test3 = l(x_test3)
y_test3.shape

In [ ]:
y_test3

### chainer.functions.softmax_cross_entropy

たとえは、[犬,猫,魚,鳥]という４つのカテゴリ（ラベル０～３）がある場合

２つの画像をネットワークにみせて、それぞれのカテゴリである確率の対数値(正規化されていない)が、[-1, 0, 1, 2]、[2, 0, 1, -1]と計算された。

In [ ]:
x_test4 = np.array([[-1, 0, 1, 2], [2, 0, 1, -1]]).astype(np.float32)
x_test4

実際の答え（教師データ）は、それぞれ、鳥（ラベル３）、犬（ラベル０）であった。

In [ ]:
t_test4 = np.array([3, 0]).astype(np.int32)
t_test4

softmax_cross_entropyにより誤差を計算します。

In [ ]:
loss_test4 = F.softmax_cross_entropy(x_test4, t_test4)
loss_test4

別のより訓練されたネットワークの答えは[-1, -1, -1, 2]、[2, -1, -1, -1]であった。

In [ ]:
x_test4_2 = np.array([[-1, -1, -1, 2], [2, -1, -1, -1]]).astype(np.float32)
x_test4_2

In [ ]:
loss_test4_2 = F.softmax_cross_entropy(x_test4_2, t_test4)
loss_test4_2

最初の例より、誤差が少なくなっていることがわかります。